In [1]:
import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm
from torch.nn import functional as F
import random
import argparse
random.seed(0)

import dataset
import model
import trainer
import utils

In [4]:
torch.tril(torch.ones(5, 5)).view(1, 1, 5, 5)

tensor([[[[1., 0., 0., 0., 0.],
          [1., 1., 0., 0., 0.],
          [1., 1., 1., 0., 0.],
          [1., 1., 1., 1., 0.],
          [1., 1., 1., 1., 1.]]]])

In [18]:
torch.zeros(100)[...,]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])

In [21]:
torch.tensor([i for i in range(10)])[...,]

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [25]:
pretrain_corpus_path = "../wiki.txt"
writing_params_path = "vanilla.model.params"
finetune_corpus_path = "../birth_places_train.tsv"

In [23]:
block_size = 128
text = open(pretrain_corpus_path, encoding="utf8").read()
pretrain_dataset = dataset.CharCorruptionDataset(text, block_size)

# We don't suggest you change these hyperparameters, as they're known to work.
# use them for both the vanilla and the synthesizer models
mconf = model.GPTConfig(pretrain_dataset.vocab_size, pretrain_dataset.block_size,
    n_layer=4, n_head=8, n_embd=256)

data has 418352 characters, 256 unique.


In [26]:
#vanilla

m = model.GPT(mconf)

fine_text = open(finetune_corpus_path, encoding="utf8").read()
train_dataset = dataset.NameDataset(pretrain_dataset, fine_text)

tconf = trainer.TrainerConfig(max_epochs=75, batch_size=256, learning_rate=6e-4,
                    lr_decay=True, warmup_tokens=512*20, final_tokens=200*len(pretrain_dataset)*block_size,
                    num_workers=4)
trainer = trainer.Trainer(m, train_dataset, None, tconf)
trainer.train()
torch.save(m.state_dict(), writing_params_path)

number of parameters: 3323392


KeyboardInterrupt: 

In [20]:
conf = model.GPTConfig(pretrain_dataset.vocab_size, pretrain_dataset.block_size,
    n_layer=4, n_head=8, n_embd=256)
m = model.GPT(conf)

AttributeError: 'GPT' object has no attribute 'GPTConfig'

In [3]:
import random

In [8]:

for _ in range(10):
    print(int(random.uniform(-2,2) + 5))

4
3
5
5
6
6
5
5
6
6


In [25]:
l = [random.uniform(-1/8, 1/8) + 5 for _ in range(1000000)]
sum(l)/len(l)

5.000047420610929

In [4]:
random.randint(1,1)

1

In [20]:
noise = random.uniform(-1/8, 1/8) * len(truncated_document) # float
masked_content_len = mean_masked_content_len + noise

6

In [21]:
random.uniform(-1/8, 1/8) * 10

-0.6283171301544757